In [ ]:
import requests
import lxml.html as lh
import pandas as pd
import csv
 
url = 'https://auvergnerhonealpes.simplon.co/candidatures.html'
uri_pages = '?page='
nbPages = 400
tags = ['//h2[@class="card-title"]', \
        '//div[@class="card-content"]', \
        '//span[@class="day"]', \
        '//span[@class="month"]', \
        '//div[@class="year"]', \
        '//div[@class="card-session-info calendar"]', \
        '//div[@class="card-session-infos"]', \
        
       ]

cols = ['Nom', 'Lieu', 'Jour', 'Mois', 'Annee', 'Debut', 'Information']
 
page = requests.get(url)
doc = lh.fromstring(page.content)
 
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
 
    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))
 
    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)
 
    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
     
    return liste
 
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale
 
liste_totale = getPages(nbPages, url)


In [ ]:
liste_totale

In [ ]:
liste_totale['Debut']

In [ ]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'event':''}, regex= True)

In [ ]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'\n':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({'Début':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({':':''}, regex= True)

In [ ]:
liste_totale

In [ ]:
liste_totale['Information'] = liste_totale['Information'].replace({'event':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'\n':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'Début':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({':':''}, regex= True)

In [ ]:
liste_totale

In [ ]:
df = liste_totale.copy()
df["duree1"] = df["Information"].str.split().str[3]
df["duree2"] = df["Information"].str.split().str[4]
df["duree3"] = df["Information"].str.split().str[5]
df["duree4"] = df["Information"].str.split().str[6]

In [ ]:
df["Localisation"] = df["Information"].str.split().str[8]

In [ ]:
df = df.drop('Information', axis=1)

In [ ]:
df['Duree'] = df['duree1'] + ' ' + df['duree2'] + ' ' + df['duree3'] + ' ' + df['duree4']

In [ ]:
df = df.drop(['duree1', 'duree2', 'duree3', 'duree4',], axis=1)

In [ ]:
df['datelimite'] = df['Jour'] + '-' + df['Mois'] + '-' + df['Annee']

In [ ]:
df = df.drop(['Jour', 'Mois', 'Annee'], axis=1)

In [ ]:
pd.set_option('display.max_rows', 56)

In [ ]:
df = df.head(57)

In [ ]:
df

In [ ]:
df.to_csv('formation_simplon.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)